# CS 1656 – Introduction to Data Science 

## Instructor: Alexandros Labrinidis / Teaching Assistant: Xiaoting Li
### Additional credits: Tahereh Arabghalizi, Evangelos Karageorgos, Zuha Agha, Anatoli Shein, Phuong Pham
## Recitation 4: SQL via Data API
---
In this recitation, you will execute SQL queries on real data by connecting to the open data portal of [Western Pennsylavnia Regional Data Center](https://www.wprdc.org/) and requesting data via API calls.  

In [ ]:
import json
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

We will be using Allegheny County Restaurant/Food Facility Inspection Violation Dataset found here https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations. This dataset contains violation data from actual routine inspections by one of health department staff's members for the last two years. It should be fun to find out inspection results for places where we eat in Pittsburgh! =)

In [ ]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

# id for database table
resource_id = "1a1329e2-418c-4bd3-af2c-cc334e7559af"

# Get the date from 9 months ago)
start_date = datetime.now() - timedelta(days=270)

# Convert to a string the format the the data center accepts (yyyy-mm-dd)
start_str = start_date.strftime("%Y-%m-%d")

# SQL query we'll use in API call to request data
query = """
SELECT *
FROM "{}"
WHERE "inspect_dt" >= '{}' AND "city" = '{}' """.format(resource_id, start_str, "Pittsburgh")

# Make WPRDC API Call
response = requests.get(wprdc_api_endpoint, {'sql': query})

# Parse response JSON into python dictionary
response_data = json.loads(response.text)

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(response_data['result']['records'])

# Print the number of rows
print(df.shape[0], "rows total")

df.head()


Details of dataset attributes are below. (Taken from https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations/resource/1a1329e2-418c-4bd3-af2c-cc334e7559af)


## Queries

__Q1) Find all unique decription categories of violation in Pittsburgh restaurants over the past nine months.__

In [ ]:
query = """
SELECT DISTINCT "description_new"
FROM "{}"
WHERE "inspect_dt" >= '{}' AND "city" = '{}' """.format(resource_id, start_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query})
df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

__Q2) Find restaurants in Pittsburgh with no violations in at least one decription category over the past nine months.__

In [ ]:
query = """
SELECT "facility_name", COUNT("description_new")
FROM "{}"
WHERE "inspect_dt" >= '{}' AND "city" = '{}' AND "rating" <> '{}'
GROUP BY "facility_name" """.format(resource_id, start_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query})

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

### Tasks

__T1) Find top 30 restaurants in Pittsburgh with maximum number of violations.__

Lets look more closely into the inspection results of Pitt facilities.

__T2) Find the category descriptions and their high, medium, low risk ratings for all violations at facilities that start with 'Pitt' over the past nine months.__

Now lets look at all facilities that contain word 'Pitt'.

__T3) Find the category descriptions and their high, medium, low risk ratings for all violations at all facilities that have word 'Pitt' in their name over the past nine months. Note that results that contain word 'Pitt' as part of another word (e.g. 'Pittsburgh') should not be included__

__T4) Find top 20 facilities that have word 'Pitt' in their name and have the highest counts of violations over the past nine months.__

Let's make a bar graph of these counts now

__T5) Create a bar graph of the counts from T4).__